# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [81]:

import requests
import os
import json
import html

In [3]:
# Had to do a workaround for my keys because env variables were being a pain while working on a remote server

prim = ""
sec = ""
with open("/home/thowe/temp-keys/tfl-primary") as f:
    prim = f.read()
    f.close()
    
    
with open("/home/thowe/temp-keys/tfl-secondary") as f:
    sec = f.read()
    f.close()

## Demo

In [4]:
# don't forget package os


# app_id = os.environ["<your app name>"]
# app_key = os.environ["<your app key name>"]

# app_id = "bikes"
app_key = sec.rstrip()

# url_append = f'?app_id={app_id}&app_key={app_key}' 
url_append = f'?app_key={app_key}' 

In [5]:
print(url_append)

?app_key=b0d6743bffd54f2da4f32935c15fd6e9


In [6]:
# URL
url = "https://api.tfl.gov.uk/AirQuality"
print(url+url_append)

https://api.tfl.gov.uk/AirQuality?app_key=b0d6743bffd54f2da4f32935c15fd6e9


In [7]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append
res = requests.get(url+url_append)
#  res = requests.get(url)

In [8]:
# We can check if the request was successful
res.status_code

200

In [9]:
# We can export the information that was returned using method .json()
res.json()

{'$id': '1',
 '$type': 'Tfl.Api.Presentation.Entities.LondonAirForecast, Tfl.Api.Presentation.Entities',
 'updatePeriod': 'hourly',
 'updateFrequency': '1',
 'forecastURL': 'http://londonair.org.uk/forecast',
 'disclaimerText': 'This forecast is intended to provide information on expected pollution levels in areas of significant public exposure. It may not apply in very specific locations close to unusually strong or short-lived local sources of pollution.',
 'currentForecast': [{'$id': '2',
   '$type': 'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
   'forecastType': 'Current',
   'forecastID': '36905',
   'fromDate': '2022-01-07T00:00:00Z',
   'toDate': '2022-01-07T23:59:00Z',
   'forecastBand': 'Low',
   'forecastSummary': 'Low air pollution forecast valid from Friday 1 July to end of Friday 1 July GMT',
   'nO2Band': 'Low',
   'o3Band': 'Low',
   'pM10Band': 'Low',
   'pM25Band': 'Low',
   'sO2Band': 'Low',
   'forecastText': 'Friday is forecast to 

In [10]:
weather_dict = res.json()

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [82]:
print(html.unescape(weather_dict['currentForecast'][1]['forecastText']).replace("<br/><br/>", " ").replace("<br/>",", ")[:-2])

Saturday is expected to be mostly dry and cloudy, although still a chance of rain. A steady feed of south westerly air arriving from the Atlantic should give good dispersion of local emissions while bearing little imported pollution.  Air pollution is expected to remain 'Low' for the following pollutants throughout the forecast period: Nitrogen Dioxide, PM10 Particulate, PM2.5 Particulate, Ozone, Sulphur Dioxide 


## Task
What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [12]:
j_modes_url = 'https://api.tfl.gov.uk/Journey/Meta/Modes'
l_modes_url = 'https://api.tfl.gov.uk/Line/Meta/Modes'

res2 = requests.get(l_modes_url + url_append)

In [83]:
modes_list = []
for mode_dict in res2.json():
    modes_list = modes_list + [mode_dict['modeName']]
    
    
print(f"There are {len(modes_list)} available modes of public transport in London.  The full list is as follows:")
for mode in modes_list:
     print(f"     • {mode.replace('-',' ').title()}")
#      print(f"   • {mode} ")


There are 18 available modes of public transport in London.  The full list is as follows:
     • Bus
     • Cable Car
     • Coach
     • Cycle
     • Cycle Hire
     • Dlr
     • Elizabeth Line
     • Interchange Keep Sitting
     • Interchange Secure
     • National Rail
     • Overground
     • Replacement Bus
     • River Bus
     • River Tour
     • Taxi
     • Tram
     • Tube
     • Walking


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [84]:
bikepoints_query_url = "https://api.tfl.gov.uk/BikePoint/"
res_bikepoints = requests.get(bikepoints_query_url + url_append)

num_bikepoints = len(res_bikepoints.json())
num_docks = 0

for bikepoint in res_bikepoints.json():
    bikepoint["AdditionalProperties"]
    
num_docks += next(bikepoint for bikepoint in res_bikepoints.json())

KeyError: 'AdditionalProperties'

In [42]:

print(f"There are {num_bikepoints} BikePoints currently operated by Transport for London")
print(f"In total, there are {} total docks in all the BikePoints in London")

There are 791 Bikepoints currently operated by Transport for London


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [74]:
tube_bus_query = "https://api.tfl.gov.uk/Line/Mode/"

bus_append = "bus"  + url_append
tube_append = "tube" + url_append

print(tube_bus_query + tube_append)

res_bus = requests.get(tube_bus_query + bus_append)
res_tube = requests.get(tube_bus_query + tube_append)


https://api.tfl.gov.uk/Line/Mode/tube?app_key=b0d6743bffd54f2da4f32935c15fd6e9


In [77]:
print(f"There are {len(res_bus.json())} bus lines in london.")
print(f"There are {len(res_tube.json())} tube lines in london.")
print("The names of the tube lines are as follows:")
for line in res_tube.json():
    print (f"   • {line['name']}") 

There are 677 bus lines in london.
There are 11 tube lines in london.
The names of the tube lines are as follows:
   • Bakerloo
   • Central
   • Circle
   • District
   • Hammersmith & City
   • Jubilee
   • Metropolitan
   • Northern
   • Piccadilly
   • Victoria
   • Waterloo & City


## Task
How many station has `victoria` line?

In [79]:
victoria_query = "https://api.tfl.gov.uk/Line/victoria/StopPoints"
res_victoria = requests.get(victoria_query + url_append);

In [26]:
res_victoria.json()
station_list = []
for station_dict in res_victoria.json():
    station_list = station_list + [station_dict["commonName"]]
    
print(f"There are {len(station_list)} stations in the Victoria line.  The list of stations is as follows:")
for station in station_list:
      print(f"   • {station} ")
    

There are 16 stations in the Victoria line.  The list of stations is as follows:
   • Blackhorse Road Underground Station 
   • Brixton Underground Station 
   • Euston Underground Station 
   • Finsbury Park Underground Station 
   • Green Park Underground Station 
   • Highbury & Islington Underground Station 
   • King's Cross St. Pancras Underground Station 
   • Oxford Circus Underground Station 
   • Pimlico Underground Station 
   • Stockwell Underground Station 
   • Seven Sisters Underground Station 
   • Tottenham Hale Underground Station 
   • Victoria Underground Station 
   • Vauxhall Underground Station 
   • Warren Street Underground Station 
   • Walthamstow Central Underground Station 


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [37]:
from_str = "920GLHR5"
to_str = "City%20of%20London,%20Tower%20Bridge"

journey_url = f"https://api.tfl.gov.uk/Journey/JourneyResults/{from_str}/to/{to_str}"
journey_tube_append = url_append + "&mode=tube"
journey_bus_append = url_append + "&mode=bus"

extra_param_append = "&journeyPreference=LeastTime&walkingSpeed=Average&walkingOptimization=True"

journey_tube_res = requests.get(journey_url + journey_tube_append + extra_param_append)
journey_bus_res = requests.get(journey_url +journey_bus_append + extra_param_append)

bus_duration = journey_bus_res.json()['journeys'][0]['duration']
tube_duration = journey_tube_res.json()['journeys'][0]['duration']




In [38]:
print(f"The bus trip will take {bus_duration // 60} hours and {bus_duration % 60 } minutes.")
print(f"The tube trip will take {tube_duration // 60} hours and {tube_duration % 60 } minutes.")

The bus trip will take 2 hours and 50 minutes.
The tube trip will take 1 hours and 21 minutes.
